# Post-Training Quantization of PyTorch models with NNCF

The goal of this tutorial is to demonstrate how to use the NNCF (Neural Network Compression Framework) 8-bit quantization in post-training mode (without the fine-tuning pipeline) to optimize a PyTorch model for the high-speed inference via OpenVINO™ Toolkit. The optimization process contains the following steps:

1. Evaluate the original model.
2. Transform the original model to a quantized one.
3. Export optimized and original models to OpenVINO IR.
4. Compare performance of the obtained `FP32` and `INT8` models.

This tutorial uses a ResNet-50 model, pre-trained on Tiny ImageNet, which contains 100000 images of 200 classes (500 for each class) downsized to 64×64 colored images. The tutorial will demonstrate that only a tiny part of the dataset is needed for the post-training quantization, not demanding the fine-tuning of the model.


> **NOTE**: This notebook requires that a C++ compiler is accessible on the default binary search path of the OS you are running the notebook.


#### Table of contents:

- [Preparations](#Preparations)
    - [Imports](#Imports)
    - [Settings](#Settings)
    - [Download and Prepare Tiny ImageNet dataset](#Download-and-Prepare-Tiny-ImageNet-dataset)
    - [Helpers classes and functions](#Helpers-classes-and-functions)
    - [Validation function](#Validation-function)
    - [Create and load original uncompressed model](#Create-and-load-original-uncompressed-model)
    - [Create train and validation DataLoaders](#Create-train-and-validation-DataLoaders)
- [Model quantization and benchmarking](#Model-quantization-and-benchmarking)
    - [I. Evaluate the loaded model](#I.-Evaluate-the-loaded-model)
    - [II. Create and initialize quantization](#II.-Create-and-initialize-quantization)
    - [III. Convert the models to OpenVINO Intermediate Representation (OpenVINO IR)](#III.-Convert-the-models-to-OpenVINO-Intermediate-Representation-(OpenVINO-IR))
    - [IV. Compare performance of INT8 model and FP32 model in OpenVINO](#IV.-Compare-performance-of-INT8-model-and-FP32-model-in-OpenVINO)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

## Preparations
[back to top ⬆️](#Table-of-contents:)


In [24]:
# Install openvino package
# %pip install -q "openvino>=2024.0.0" torch torchvision tqdm --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -q "openvino>=2024.0.0" torch torchvision tqdm
%pip install -q "nncf>=2.9.0"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports
[back to top ⬆️](#Table-of-contents:)


In [25]:
import os
import time
import zipfile
from pathlib import Path
from typing import List, Tuple

import nncf
import openvino as ov

import torch
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
import torchvision.transforms as transforms

# Fetch `notebook_utils` module
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)

open("notebook_utils.py", "w").write(r.text)
from notebook_utils import download_file

### Settings
[back to top ⬆️](#Table-of-contents:)


In [26]:
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {torch_device} device")

MODEL_DIR = Path("model")
OUTPUT_DIR = Path("output")
BASE_MODEL_NAME = "resnet50"
IMAGE_SIZE = [64, 64]

OUTPUT_DIR.mkdir(exist_ok=True)
MODEL_DIR.mkdir(exist_ok=True)

# Paths where PyTorch and OpenVINO IR models will be stored.
fp32_checkpoint_filename = Path(BASE_MODEL_NAME + "_fp32").with_suffix(".pth")
fp32_ir_path = OUTPUT_DIR / Path(BASE_MODEL_NAME + "_fp32").with_suffix(".xml")
int8_ir_path = OUTPUT_DIR / Path(BASE_MODEL_NAME + "_int8").with_suffix(".xml")


fp32_pth_url = "https://storage.openvinotoolkit.org/repositories/nncf/openvino_notebook_ckpts/304_resnet50_fp32.pth"
download_file(fp32_pth_url, directory=MODEL_DIR, filename=fp32_checkpoint_filename)

Using cpu device
'model/resnet50_fp32.pth' already exists.


PosixPath('/data/xkd_data/8-SourceCode/Viettel/OpenVINO-Hands-On/model/resnet50_fp32.pth')

### Download and Prepare Tiny ImageNet dataset
[back to top ⬆️](#Table-of-contents:)

* 100k images of shape 3x64x64,
* 200 different classes: snake, spider, cat, truck, grasshopper, gull, etc.

In [27]:
def download_tiny_imagenet_200(
    output_dir: Path,
    url: str = "http://cs231n.stanford.edu/tiny-imagenet-200.zip",
    tarname: str = "tiny-imagenet-200.zip",
):
    archive_path = output_dir / tarname
    if not os.path.exists(archive_path):
        download_file(url, directory=output_dir, filename=tarname)
    zip_ref = zipfile.ZipFile(archive_path, "r")
    zip_ref.extractall(path=output_dir)
    zip_ref.close()
    print(f"Successfully downloaded and extracted dataset to: {output_dir}")


def create_validation_dir(dataset_dir: Path):
    VALID_DIR = dataset_dir / "val"
    val_img_dir = VALID_DIR / "images"

    fp = open(VALID_DIR / "val_annotations.txt", "r")
    data = fp.readlines()

    val_img_dict = {}
    for line in data:
        words = line.split("\t")
        val_img_dict[words[0]] = words[1]
    fp.close()

    for img, folder in val_img_dict.items():
        newpath = val_img_dir / folder
        if not newpath.exists():
            os.makedirs(newpath)
        if (val_img_dir / img).exists():
            os.rename(val_img_dir / img, newpath / img)


DATASET_DIR = OUTPUT_DIR / "tiny-imagenet-200"
if not DATASET_DIR.exists():
    download_tiny_imagenet_200(OUTPUT_DIR)
    create_validation_dir(DATASET_DIR)

### Helpers classes and functions
[back to top ⬆️](#Table-of-contents:)

The code below will help to count accuracy and visualize validation process.

In [28]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name: str, fmt: str = ":f"):
        self.name = name
        self.fmt = fmt
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    """Displays the progress of validation process"""

    def __init__(self, num_batches: int, meters: List[AverageMeter], prefix: str = ""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch: int):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print("\t".join(entries))

    def _get_batch_fmtstr(self, num_batches: int):
        num_digits = len(str(num_batches // 1))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"


def accuracy(output: torch.Tensor, target: torch.Tensor, topk: Tuple[int] = (1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))

        return res

### Validation function
[back to top ⬆️](#Table-of-contents:)


In [29]:
from typing import Union
from openvino.runtime.ie_api import CompiledModel


def validate(
    val_loader: torch.utils.data.DataLoader,
    model: Union[torch.nn.Module, CompiledModel],
):
    """Compute the metrics using data from val_loader for the model"""
    batch_time = AverageMeter("Time", ":3.3f")
    top1 = AverageMeter("Acc@1", ":2.2f")
    top5 = AverageMeter("Acc@5", ":2.2f")
    progress = ProgressMeter(len(val_loader), [batch_time, top1, top5], prefix="Test: ")
    start_time = time.time()
    # Switch to evaluate mode.
    if not isinstance(model, CompiledModel):
        model.eval()
        model.to(torch_device)

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(torch_device)
            target = target.to(torch_device)

            # Compute the output.
            if isinstance(model, CompiledModel):
                output_layer = model.output(0)
                output = model(images)[output_layer]
                output = torch.from_numpy(output)
            else:
                output = model(images)

            # Measure accuracy and record loss.
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # Measure elapsed time.
            batch_time.update(time.time() - end)
            end = time.time()

            print_frequency = 10
            if i % print_frequency == 0:
                progress.display(i)

        print(" * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f} Total time: {total_time:.3f}".format(top1=top1, top5=top5, total_time=end - start_time))
    return top1.avg

### Create and load original uncompressed model
[back to top ⬆️](#Table-of-contents:)

ResNet-50 from the [`torchivision` repository](https://github.com/pytorch/vision) is pre-trained on ImageNet with more prediction classes than Tiny ImageNet, so the model is adjusted by swapping the last FC layer to one with fewer output values.

In [30]:
def create_model(model_path: Path):
    """Creates the ResNet-50 model and loads the pretrained weights"""
    model = resnet50()
    # Update the last FC layer for Tiny ImageNet number of classes.
    NUM_CLASSES = 200
    model.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASSES, bias=True)
    model.to(torch_device)
    if model_path.exists():
        checkpoint = torch.load(str(model_path), map_location="cpu")
        model.load_state_dict(checkpoint["state_dict"], strict=True)
    else:
        raise RuntimeError("There is no checkpoint to load")
    return model


model = create_model(MODEL_DIR / fp32_checkpoint_filename)

### Create train and validation DataLoaders
[back to top ⬆️](#Table-of-contents:)


In [31]:
def create_dataloaders(batch_size: int = 128):
    """Creates train dataloader that is used for quantization initialization and validation dataloader for computing the model accruacy"""
    train_dir = DATASET_DIR / "train"
    val_dir = DATASET_DIR / "val" / "images"
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    train_dataset = ImageFolder(
        train_dir,
        transforms.Compose(
            [
                transforms.Resize(IMAGE_SIZE),
                transforms.ToTensor(),
                normalize,
            ]
        ),
    )
    val_dataset = ImageFolder(
        val_dir,
        transforms.Compose([transforms.Resize(IMAGE_SIZE), transforms.ToTensor(), normalize]),
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        sampler=None,
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
    )
    return train_loader, val_loader


train_loader, val_loader = create_dataloaders()

## Model quantization and benchmarking
[back to top ⬆️](#Table-of-contents:)

With the validation pipeline, model files, and data-loading procedures for model calibration now prepared, it's time to proceed with the actual post-training quantization using NNCF.

### I. Evaluate the loaded model
[back to top ⬆️](#Table-of-contents:)


In [32]:
acc1 = validate(val_loader, model)
print(f"Test accuracy of FP32 model: {acc1:.3f}")

Test: [ 0/79]	Time 0.482 (0.482)	Acc@1 81.25 (81.25)	Acc@5 92.19 (92.19)
Test: [10/79]	Time 0.450 (0.458)	Acc@1 56.25 (66.97)	Acc@5 86.72 (87.50)
Test: [20/79]	Time 0.454 (0.457)	Acc@1 67.97 (64.29)	Acc@5 85.16 (87.35)
Test: [30/79]	Time 0.449 (0.455)	Acc@1 53.12 (62.37)	Acc@5 77.34 (85.33)
Test: [40/79]	Time 0.454 (0.455)	Acc@1 67.19 (60.86)	Acc@5 90.62 (84.51)
Test: [50/79]	Time 0.449 (0.455)	Acc@1 60.16 (60.80)	Acc@5 88.28 (84.42)
Test: [60/79]	Time 0.450 (0.454)	Acc@1 66.41 (60.46)	Acc@5 86.72 (83.79)
Test: [70/79]	Time 0.449 (0.454)	Acc@1 52.34 (60.21)	Acc@5 80.47 (83.33)
 * Acc@1 60.740 Acc@5 83.960 Total time: 35.474
Test accuracy of FP32 model: 60.740


### II. Create and initialize quantization
[back to top ⬆️](#Table-of-contents:)

NNCF enables post-training quantization by adding the quantization layers into the model graph and then using a subset of the training dataset to initialize the parameters of these additional quantization layers. The framework is designed so that modifications to your original training code are minor. Quantization is the simplest scenario and requires a few modifications.
For more information about NNCF Post Training Quantization (PTQ) API, refer to the [Basic Quantization Flow Guide](https://docs.openvino.ai/2024/openvino-workflow/model-optimization-guide/quantizing-models-post-training/basic-quantization-flow.html).

1. Create a transformation function that accepts a sample from the dataset and returns data suitable for model inference. This enables the creation of an instance of the nncf.Dataset class, which represents the calibration dataset (based on the training dataset) necessary for post-training quantization.

In [33]:
def transform_fn(data_item):
    images, _ = data_item
    return images


calibration_dataset = nncf.Dataset(train_loader, transform_fn)

2. Create a quantized model from the pre-trained `FP32` model and the calibration dataset.

In [34]:
quantized_model = nncf.quantize(model, calibration_dataset)

Output()

Output()

3. Evaluate the new model on the validation set after initialization of quantization. The accuracy should be close to the accuracy of the floating-point `FP32` model for a simple case like the one being demonstrated now.

In [35]:
acc1 = validate(val_loader, quantized_model)
print(f"Accuracy of initialized INT8 model: {acc1:.3f}")

Test: [ 0/79]	Time 0.948 (0.948)	Acc@1 80.47 (80.47)	Acc@5 91.41 (91.41)
Test: [10/79]	Time 0.887 (0.892)	Acc@1 52.34 (66.34)	Acc@5 86.72 (87.64)
Test: [20/79]	Time 0.919 (0.892)	Acc@1 68.75 (63.65)	Acc@5 85.94 (87.31)
Test: [30/79]	Time 0.886 (0.893)	Acc@1 50.78 (61.97)	Acc@5 74.22 (85.13)
Test: [40/79]	Time 0.882 (0.889)	Acc@1 67.97 (60.52)	Acc@5 89.84 (84.26)
Test: [50/79]	Time 0.884 (0.888)	Acc@1 58.59 (60.43)	Acc@5 87.50 (84.10)
Test: [60/79]	Time 0.883 (0.888)	Acc@1 65.62 (60.19)	Acc@5 85.94 (83.54)
Test: [70/79]	Time 0.888 (0.887)	Acc@1 53.12 (59.94)	Acc@5 79.69 (83.16)
 * Acc@1 60.480 Acc@5 83.760 Total time: 69.436
Accuracy of initialized INT8 model: 60.480


It should be noted that the inference time for the quantized PyTorch model is longer than that of the original model, as fake quantizers are added to the model by NNCF. However, the model's performance will significantly improve when it is in the OpenVINO Intermediate Representation (IR) format.

### III. Convert the models to OpenVINO Intermediate Representation (OpenVINO IR)
[back to top ⬆️](#Table-of-contents:)

To convert the Pytorch models to OpenVINO IR, use Model Conversion Python API. The models will be saved to the 'OUTPUT' directory for later benchmarking.

For more information about model conversion, refer to this [page](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html).


In [36]:
dummy_input = torch.randn(128, 3, *IMAGE_SIZE)

model_ir = ov.convert_model(model, example_input=dummy_input, input=[-1, 3, *IMAGE_SIZE])

ov.save_model(model_ir, fp32_ir_path)

In [37]:
quantized_model_ir = ov.convert_model(quantized_model, example_input=dummy_input, input=[-1, 3, *IMAGE_SIZE])

ov.save_model(quantized_model_ir, int8_ir_path)

/data/xkd_data/8-SourceCode/Viettel/.venv/lib/python3.10/site-packages/nncf/torch/quantization/layers.py:339: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return self._level_low.item()
/data/xkd_data/8-SourceCode/Viettel/.venv/lib/python3.10/site-packages/nncf/torch/quantization/layers.py:347: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return self._level_high.item()
/data/xkd_data/8-SourceCode/Viettel/.venv/lib/python3.10/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output

Select inference device for OpenVINO

In [38]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', index=3, options=('CPU', 'GPU.0', 'GPU.1', 'AUTO'), value='AUTO')

Evaluate the FP32 and INT8 models.

In [39]:
core = ov.Core()
fp32_compiled_model = core.compile_model(model_ir, device.value)
acc1 = validate(val_loader, fp32_compiled_model)
print(f"Accuracy of FP32 IR model: {acc1:.3f}")

Test: [ 0/79]	Time 0.332 (0.332)	Acc@1 81.25 (81.25)	Acc@5 92.19 (92.19)
Test: [10/79]	Time 0.281 (0.279)	Acc@1 56.25 (66.97)	Acc@5 86.72 (87.50)
Test: [20/79]	Time 0.266 (0.277)	Acc@1 67.97 (64.29)	Acc@5 85.16 (87.35)
Test: [30/79]	Time 0.267 (0.276)	Acc@1 53.12 (62.37)	Acc@5 77.34 (85.33)
Test: [40/79]	Time 0.274 (0.273)	Acc@1 67.19 (60.86)	Acc@5 90.62 (84.51)
Test: [50/79]	Time 0.263 (0.271)	Acc@1 60.16 (60.80)	Acc@5 88.28 (84.42)
Test: [60/79]	Time 0.268 (0.270)	Acc@1 66.41 (60.46)	Acc@5 86.72 (83.79)
Test: [70/79]	Time 0.255 (0.270)	Acc@1 52.34 (60.21)	Acc@5 80.47 (83.33)
 * Acc@1 60.740 Acc@5 83.960 Total time: 21.153
Accuracy of FP32 IR model: 60.740


In [40]:
int8_compiled_model = core.compile_model(quantized_model_ir, device.value)
acc1 = validate(val_loader, int8_compiled_model)
print(f"Accuracy of INT8 IR model: {acc1:.3f}")

Test: [ 0/79]	Time 0.258 (0.258)	Acc@1 80.47 (80.47)	Acc@5 91.41 (91.41)
Test: [10/79]	Time 0.217 (0.222)	Acc@1 53.12 (66.48)	Acc@5 86.72 (87.71)
Test: [20/79]	Time 0.221 (0.221)	Acc@1 70.31 (63.95)	Acc@5 85.16 (87.35)
Test: [30/79]	Time 0.218 (0.220)	Acc@1 50.78 (62.30)	Acc@5 74.22 (85.18)
Test: [40/79]	Time 0.213 (0.220)	Acc@1 68.75 (60.94)	Acc@5 89.84 (84.28)
Test: [50/79]	Time 0.220 (0.219)	Acc@1 60.16 (60.83)	Acc@5 87.50 (84.16)
Test: [60/79]	Time 0.215 (0.219)	Acc@1 65.62 (60.46)	Acc@5 85.94 (83.63)
Test: [70/79]	Time 0.215 (0.219)	Acc@1 52.34 (60.19)	Acc@5 78.91 (83.20)
 * Acc@1 60.700 Acc@5 83.790 Total time: 17.129
Accuracy of INT8 IR model: 60.700


### IV. Compare performance of INT8 model and FP32 model in OpenVINO
[back to top ⬆️](#Table-of-contents:)

Finally, measure the inference performance of the `FP32` and `INT8` models, using [Benchmark Tool](https://docs.openvino.ai/2024/learn-openvino/openvino-samples/benchmark-tool.html) - an inference performance measurement tool in OpenVINO. By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput (frames per second) values.

> **NOTE**: This notebook runs benchmark_app for 15 seconds to give a quick indication of performance. For more accurate performance, it is recommended to run benchmark_app in a terminal/command prompt after closing other applications. Run `benchmark_app -m model.xml -d CPU` to benchmark async inference on CPU for one minute. Change CPU to GPU to benchmark on GPU. Run `benchmark_app --help` to see an overview of all command-line options.

In [41]:
device

Dropdown(description='Device:', index=3, options=('CPU', 'GPU.0', 'GPU.1', 'AUTO'), value='AUTO')

In [42]:
def parse_benchmark_output(benchmark_output: str):
    """Prints the output from benchmark_app in human-readable format"""
    parsed_output = [line for line in benchmark_output if "FPS" in line]
    print(*parsed_output, sep="\n")


print("Benchmark FP32 model (OpenVINO IR)")
benchmark_output = ! benchmark_app -m "$fp32_ir_path" -d $device.value -api async -t 15 -shape "[1, 3, 512, 512]"
parse_benchmark_output(benchmark_output)

print("Benchmark INT8 model (OpenVINO IR)")
benchmark_output = ! benchmark_app -m "$int8_ir_path" -d $device.value -api async -t 15 -shape "[1, 3, 512, 512]"
parse_benchmark_output(benchmark_output)

print("Benchmark FP32 model (OpenVINO IR) synchronously")
benchmark_output = ! benchmark_app -m "$fp32_ir_path" -d $device.value -api sync -t 15 -shape "[1, 3, 512, 512]"
parse_benchmark_output(benchmark_output)

print("Benchmark INT8 model (OpenVINO IR) synchronously")
benchmark_output = ! benchmark_app -m "$int8_ir_path" -d $device.value -api sync -t 15 -shape "[1, 3, 512, 512]"
parse_benchmark_output(benchmark_output)

Benchmark FP32 model (OpenVINO IR)
[ INFO ] Throughput:   12.10 FPS
Benchmark INT8 model (OpenVINO IR)
[ INFO ] Throughput:   15.19 FPS
Benchmark FP32 model (OpenVINO IR) synchronously
[ INFO ] Throughput:   11.80 FPS
Benchmark INT8 model (OpenVINO IR) synchronously
[ INFO ] Throughput:   14.41 FPS


Show device Information for reference:

In [43]:
core = ov.Core()
devices = core.available_devices

for device_name in devices:
    device_full_name = core.get_property(device_name, "FULL_DEVICE_NAME")
    if not "NVIDIA" in device_full_name:
        print(f"{device_name}: {device_full_name}")

CPU: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz
